In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
import glob
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.utils import load_img, img_to_array

In [5]:
#Define the shape for the test inputs

image_size = (50, 50)
input_shape = (None,) + image_size + (3,)
batch_size = 32

In [6]:
#Loading the test data
test_dir = pathlib.Path('Data/test')

In [7]:
#Creating the test dataset

test_ds = image_dataset_from_directory(
    test_dir, 
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    seed = 42,
)

Found 539 files belonging to 3 classes.


In [8]:
#Loading trained model
final_model = keras.models.load_model('shape_50_loss_0.6965.keras')

final_model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_20 (Sequential)  (None, None, None, 3)     0         
                                                                 
 conv2d_110 (Conv2D)         (None, 50, 50, 128)       3584      
                                                                 
 batch_normalization_110 (Ba  (None, 50, 50, 128)      512       
 tchNormalization)                                               
                                                                 
 re_lu_110 (ReLU)            (None, 50, 50, 128)       0         
                                                                 
 max_pooling2d_110 (MaxPooli  (None, 25, 25, 128)      0         
 ng2D)                                                           
                                                                 
 conv2d_111 (Conv2D)         (None, 25, 25, 128)     

In [9]:
#Evaluating model on test set

final_model.evaluate(test_ds)

17/17 [==============================] - 1s 40ms/step - loss: 0.6549 - accuracy: 0.8219


[0.654863178730011, 0.8218923807144165]

In [10]:
#Extract class names for sample predictions
class_names = test_ds.class_names


In [20]:
#Loading the specified test samples

sample_crack = pathlib.Path('Data/test')

img = load_img(sample_crack, target_size=(50, 50))

#def preprocess_images(image_path, image_size):


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\ahadr\\Desktop\\Aerospace Engineering\\F2024\\AER 850\\Project-2\\Data\\test'